* Think of a word or words that define each point  
    * Proximity to each of those words creates a new centroid.
    * proximity is determined using pre-trained word2vec
    * basically, check how close a new word is to each of the centroids (which are defined by words)
    * how close it is to each centroid contributes contributes to it's distance in that direction

In [1]:
import pandas as pd
import numpy as np

import pickle

from scipy.spatial import minkowski_distance

Find songs closest to each centroid and define those moods

In [2]:
pickle_in = open('../Data/cluster_centers_tfidf_pca_12', 'rb')
centroids = pickle.load(pickle_in)
pickle_in.close()

In [3]:
df = pd.read_pickle('../Data/labeled_points_tf-idf_pca_12')

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [5]:
features = df.select_dtypes(include=numerics)
no_features = df.select_dtypes(exclude=numerics)

In [6]:
centroids[0]

array([-0.28504324, -0.25274528,  0.01268743,  1.04607193, -0.50220715,
       -1.08316163,  2.29063861, -0.254587  , -1.38358669, -0.20955775,
       -0.86723379])

In [7]:
def match_sentiment(sentiment_df, feats_to_match):
    distances_df = sentiment_df.apply(lambda x:
                                      minkowski_distance(x.values,
                                                         feats_to_match, p=1),
                                      axis=1)
    distances_df = pd.DataFrame(distances_df)
    distances_df.columns = ['distance']
    n_smallest = distances_df.nsmallest(15, 'distance').index
    return n_smallest

In [8]:
n_smallest = match_sentiment(features.drop('labels', axis=1), centroids[4])

In [9]:
df[df.labels == 4].shape

(2171, 31)

In [10]:
for i in n_smallest:
    print(df.iloc[i].uri + ',',
          df.iloc[i].unaltered_artist_name + ',',
          df.iloc[i].unaltered_song_title,
          '\n')

spotify:track:3KsPvaipGbElEhQbLCN2CF, Voxtrot, Kid Gloves 

spotify:track:12p1qNp68XkzfoezXpGS6z, Echosmith, Talking Dreams 

spotify:track:0Nienbwjw9DT60GG2r0Vda, Imogen Heap, Angry Angel 

spotify:track:3PseAA6dNJjRrsOnmxYw7R, Ballboy, Where Do The Nights Of Sleep Go To When They Do Not Come To Me 

spotify:track:3FaB0W0rslVptDbgWsrUVa, Dr. Dog, Warrior Man 

spotify:track:5P42x99XKrdBPMUJJTBWku, The Chameleons, Don't Fall 

spotify:track:7IzzbtXswTcAXo0EzQHmTe, The Whitlams, I Will Not Go Quietly 

spotify:track:5kcySB0rKK9MANsHAfQuET, Tokyo Police Club, Breakneck Speed 

spotify:track:7jOBnNKTGY10qWnldm1cZ1, Nada Surf, Mother's Day 

spotify:track:4ZYiSQSGnE8FeDl2j2RKuD, The Car Is On Fire, North By Northwest 

spotify:track:259vWMFpCxCu3Bc7Ts4Yvo, Let's Go Sailing, Heart Condition 

spotify:track:1WQ3pFFsvBVLoh0UVWsSur, Ben Kweller, No Reason 

spotify:track:3vRO5zFrSk8BRYq1CYzyvC, The Most Serene Republic, Relative's Eyes 

spotify:track:0fj0UQDgxAZmPyatxa0noB, Broder Daniel, Dum

Centroid descriptions:
* 0:
    * psychedelic 
    * raw
    * distant
    * trippy
    * weird
    * lost
    * on my own path
    * outside/wild
    * primal
    * carnal
    * appreciating simple things, simple life
    * country
    * acoustic
* 1:
    * agitated
    * frustrated
    * annoyed
    * very lost
    * mad at the world
    * sad x 2
    * alone x 2
    * sad 
    * lonely
* 2:
    * quiet
    * content
    * let go
    * etherial
    * regret
    * missing old times
    * I feel like I've lost someone
    * letting go
    * having trouble letting go
* 3:
    * no idea... skipping for now
* 4:


In [23]:
df[(df.artist_name == 'esthero') & (df.song_title == 'crash prelude')].lyrics.values

array(['crash floor hold breath fall apart made simple mistake paying heart speak bed afford lay remember always said leave alone cause wanna crash afraid going nowhere way fast hear say conversation past maybe brave much raising got somehow path paved keep leading wanna crash wanna crash'],
      dtype=object)

In [23]:
def calculate_distances(sentiment_df, feats_to_match):
    distances_df = sentiment_df.apply(lambda x:
                                      minkowski_distance(x.values,
                                                         feats_to_match, p=1),
                                      axis=1)
    distances_df = pd.DataFrame(distances_df)
    distances_df.columns = ['distance']

    # We sort by distance and reset the index. We then choose songs based on
    # index. Smaller indices are more probable.
    
    distances_df = distances_df.sort_values('distance').reset_index(drop=True)

    lst = []
    while len(lst) < 10:
        num = int(np.random.exponential(scale=15))
        if num not in np.array(lst):
            lst.append(num)

    return lst, distances_df

In [51]:
n_smallest = match_sentiment(features.drop('labels', axis=1), centroids[7])

In [52]:
for i in n_smallest:
    print(df.iloc[i].uri + ',',
          df.iloc[i].unaltered_artist_name + ',',
          df.iloc[i].unaltered_song_title,
          '\n')

spotify:track:5H5cNU45KyPXMlrxTMj2XM, Zero 7, Left Behind 

spotify:track:2tDdK0BXmN1cnqZEEferZU, Zero 7, Futures 

spotify:track:0gBTPgvzYPMPbYlTuPjx9n, Zero 7, In The Waiting Line 

spotify:track:3sgqFOFsyW9B8UnQu1hNuw, Zero 7, Take Me Away 

spotify:track:0QvF5Q3dg3TpRsU6Wqz9EV, Zero 7, This World 

spotify:track:2C5Rn73NliSEmFy6xCZjbg, Zero 7, Swing 

spotify:track:3G5qMSP9AywIErX4HSbP5a, Zero 7, Spinning 

spotify:track:2YAHNqJ5Ta5TbSuHMVe3Mt, Zero 7, Simple Science 

spotify:track:0naBEjO1WJ90v3qAzgdEVe, Zero 7, The Road 

spotify:track:5wSZYOGHOuv4GtIDejqERy, Zero 7, Everything Up [Zizou] 



In [47]:
lst = []
while len(lst) < 10:
    num = int(np.random.exponential(scale=10))
    if num not in np.array(lst):
        lst.append(num)
lst

[7, 10, 11, 3, 1, 46, 2, 14, 5, 35]